import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
**EDA for NHL Game Data**

# Imports 

In [320]:
import mysql.connector
from mysql.connector.constants import ClientFlag
import os
import json
import requests
import pprint

# Connecting to Database
https://towardsdatascience.com/sql-on-the-cloud-with-python-c08a30807661

In [326]:
config = {
    'user': 'root',
    'password': 'dyfNaj-6tozri-nemjej',
    'host': '34.86.62.110',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': '../other/ssl/server-ca.pem',
    'ssl_cert': '../other/ssl/client-cert.pem',
    'ssl_key': '../other/ssl/client-key.pem',
    'database': 'nhldatabase'
}

In [327]:
# now we establish our connection
cnxn = mysql.connector.connect(**config)

In [328]:
assert cnxn

## Examples of how to use the database

In [ ]:
"""
cursor = cnxn.cursor()  # initialize connection cursor
cursor.execute('CREATE DATABASE testdb')  # create a new 'testdb' database
cnxn.commit()  # this commits changes to the database
cnxn.close()  # close connection because we will be reconnecting to testdb
"""

In [ ]:
# to get data from database (i.e.: run a query)
"""
cursor.execute(<SQL query in quotes>)
<variable> cursor.fetchall() # i guess this performs the query
# view the data in the variable
"""

# Getting General Info about database

In [324]:
assert cnxn

In [322]:
def run_query(sql_query, connection):
    # add try except for connection
    try:
        assert connection
    except AssertionError:
        connection = mysql.connector.connect(**config)
    # main part of function
    cursor = connection.cursor()
    cursor.execute(sql_query)
    variable = cursor.fetchall()
    return variable

## Tables

In [88]:
test_query = "SHOW TABLES;"
all_tables_query_result = run_query(test_query, cnxn)
all_tables = [a[0] for a in all_tables_query_result]

In [89]:
all_tables # ignoring entries

['entries',
 'game',
 'game_goalie_stats',
 'game_plays',
 'game_plays_players',
 'game_shifts',
 'game_skater_stats',
 'game_teams_stats',
 'player_info',
 'team_info']

## Type of events covered in tables 

In [72]:
game_plays_query = """
    SHOW columns FROM game_plays
""" # https://stackoverflow.com/questions/4165195/mysql-query-to-get-column-names
columns_in_game_plays = run_query(game_plays_query, cnxn)

In [90]:
first_5_game_plays_query = """
    SELECT *
    FROM game_plays
    LIMIT 5;
"""
first_5_game_plays = run_query(first_5_game_plays_query, cnxn)

In [91]:
first_5_game_plays[1]

('2011030221_2',
 2011030221,
 2,
 0.0,
 0.0,
 'Period Ready',
 'NA',
 0.0,
 0.0,
 1,
 'REGULAR',
 0,
 1200,
 '2012-04-29 19:19:49',
 0,
 0,
 'Period Ready',
 0.0,
 0.0,
 'NA\r')

## Making all columns in all tables into a dataframe

In [93]:
all_query_results = []
for table in all_tables[1:]:
    query = f"""
                SHOW columns FROM {table}
            """
    query_result = run_query(query, cnxn) # returns:  Field       | Type     | Null | Key | Default | Extra     
    table_w_query = {table: query_result}
    all_query_results.append(table_w_query)

In [149]:
# get all columns in a contingency table 
all_columns = [] 
for thing in all_query_results:
    for key, value in thing.items():
        for item in value:
            list_item = list(item)
            list_item.append(key)
    #         one_col.append(key) # values (and its features) are first, table is last
            all_columns.append(list_item)

In [158]:
# removing incorrect values
for x in all_columns:
    if len(x) != 7:
        all_columns.remove(x)

In [172]:
table_info_df = pd.DataFrame(data = all_columns, 
             columns = ["Column_name", "Column_type", "Null", "Key", "Default", "Extra", "Table"])


<IPython.core.display.Javascript object>

In [173]:
# key, null and extra are all the same value, so I'm removing them
table_info_df.drop(columns = ['Null', "Key", "Extra", "Default"], inplace=True)

In [174]:
table_info_df.head()

,Column_name,Column_type,Table
0,game_id,int(11),game
1,season,int(11),game
2,type,varchar(1),game
3,date_time,varchar(10),game
4,date_time_GMT,varchar(20),game


### Adding num of unique values to df

In [176]:
num_unique_values_list = []
for idx in range(len(table_info_df)):
    col_name = table_info_df['Column_name'][idx]
    table_name = table_info_df['Table'][idx]
    query = f"""
            SELECT COUNT(DISTINCT({col_name}))
            FROM {table_name}
    """
    query_result = run_query(query, cnxn)
    num_unique_values_list.append(query_result)

In [183]:
# flattening the num_unique_values_list
flattened_num_unique_values = [num_unique_values_list[x][0][0] for x in range(len(num_unique_values_list))]

In [185]:
table_info_df['Number_of_unique_values'] = flattened_num_unique_values

In [316]:
# table_info_df[0:60]
table_info_df[60:]

,Column_name,Column_type,Table,Number_of_unique_values
60,game_id,int(11),game_shifts,11339
61,player_id,int(11),game_shifts,2212
62,period,int(11),game_shifts,7
63,shift_start,int(11),game_shifts,5911
64,shift_end,int(11),game_shifts,5913
65,game_id,int(11),game_skater_stats,11435
66,player_id,int(11),game_skater_stats,1999
67,team_id,int(11),game_skater_stats,34
68,timeOnIce,int(11),game_skater_stats,2220
69,assists,int(11),game_skater_stats,6


In [330]:
position_query = """
    SELECT DISTINCT(primaryPosition)
        FROM player_info
"""
positions = run_query(position_query, cnxn)
positions

[('pr',), ('D',), ('C',), ('LW',), ('RW',), ('G',)]

In [215]:
# what events are possible
events_query = """
        SELECT DISTINCT(event)
        FROM game_plays
"""
all_events = run_query(events_query, cnxn)
# all_events

[('event',),
 ('Period Ready',),
 ('Period Start',),
 ('Faceoff',),
 ('Blocked Shot',),
 ('Hit',),
 ('Stoppage',),
 ('Shot',),
 ('Giveaway',),
 ('Penalty',),
 ('Takeaway',),
 ('Missed Shot',),
 ('Period End',),
 ('Period Official',),
 ('Goal',),
 ('Game End',),
 ('Game Scheduled',),
 ('Game Official',),
 ('Official Challenge',),
 ('Shootout Complete',),
 ('Early Intermission Start',),
 ('Early Intermission End',),
 ('Emergency Goaltender',)]

In [231]:
secondaryevents_query = """
        SELECT DISTINCT(secondaryType)
        FROM game_plays
"""
all_secondaryevents = run_query(secondaryevents_query, cnxn)
all_secondaryevents # deflected/tip-in

[('secondaryType',),
 ('NA',),
 ('Wrist Shot',),
 ('Snap Shot',),
 ('Deflected',),
 ('Roughing',),
 ('Hooking',),
 ('Holding',),
 ('Slap Shot',),
 ('Delaying Game - Puck over glass',),
 ('Backhand',),
 ('Wrap-around',),
 ('Tip-In',),
 ('Misconduct',),
 ('Slashing',),
 ('Elbowing',),
 ('Tripping',),
 ('Hi-sticking',),
 ('Interference - Goalkeeper',),
 ('Boarding',),
 ('Interference',),
 ('Cross checking',),
 ('Fighting',),
 ('Too many men on the ice',),
 ('Charging',),
 ('Hi stick - double minor',),
 ('Holding the stick',),
 ('Unsportsmanlike conduct',),
 ('Checking from behind',),
 ('Game misconduct',),
 ('PS - Slash on breakaway',),
 ('Closing hand on puck',),
 ('Instigator',),
 ('Instigator - face shield',),
 ('Face-off violation',),
 ('Diving',),
 ('PS - Tripping on breakaway',),
 ('Kneeing',),
 ('Illegal check to head',),
 ('Delay of game',),
 ('Match penalty',),
 ('Delaying the game',),
 ('Abusive language',),
 ('Embellishment',),
 ('Delay Gm - Face-off Violation',),
 ('PS - Hooki

In [203]:
# how does datetime work in game_plays
## period time works much better 
datetime_query = """
        SELECT dateTime
        FROM game_plays
        LIMIT 10
"""
datetime_10 = run_query(datetime_query, cnxn)
datetime_10

[('dateTime',),
 ('2012-04-29 19:19:49',),
 ('2012-04-29 19:20:13',),
 ('2012-04-29 19:20:13',),
 ('2012-04-29 19:21:07',),
 ('2012-04-29 19:21:40',),
 ('2012-04-29 19:22:42',),
 ('2012-04-29 19:22:59',),
 ('2012-04-29 19:23:44',),
 ('2012-04-29 19:24:23',)]

In [211]:
# periodtime works in seconds within period
periodtime_query = """
        SELECT periodTime
        FROM game_plays
        LIMIT 10
        OFFSET 250
"""
periodtime_10 = run_query(periodtime_query, cnxn)
periodtime_10

[(1028,),
 (1028,),
 (1043,),
 (1064,),
 (1084,),
 (1096,),
 (1096,),
 (1149,),
 (1200,),
 (1200,)]

In [218]:
# game_plays description...what's that about
description_query = """
        SELECT description
        FROM game_plays
        LIMIT 10
        Offset 1500
"""
description_10 = run_query(description_query, cnxn)
description_10

[('Andy McDonald faceoff won against Anze Kopitar',),
 ('Anze Kopitar faceoff won against David Backes',),
 ('David Backes blocked shot from Rob Scuderi',),
 ('Vladimir Sobotka hit Dustin Penner',),
 ('Mike Richards Deflected saved by Brian Elliott',),
 ('Kevin Shattenkirk Roughing against Mike Richards',),
 ('Alec Martinez Slap Shot saved by Brian Elliott',),
 ('Offside',),
 ('Jarret Stoll faceoff won against David Backes',),
 ('Mike Richards faceoff won against Patrik Berglund',)]

In [239]:
134 % 100

34

In [235]:
len(str(134))

3

In [237]:
2 % 100

2

In [240]:
# what seasons does this cover
seasons_query = """
        SELECT DISTINCT (season)
        FROM game
"""
distinct_seasons = run_query(seasons_query, cnxn)
distinct_seasons

[(0,),
 (20112012,),
 (20102011,),
 (20122013,),
 (20162017,),
 (20142015,),
 (20152016,),
 (20132014,),
 (20172018,),
 (20182019,)]

In [242]:
# getting a game id to put into nhl API 
gameid_query = """
        SELECT DISTINCT (game_id)
        FROM game
        LIMIT 10
"""
example_game_ids = run_query(gameid_query, cnxn)
example_game_ids

[(0,),
 (2011030222,),
 (2011030224,),
 (2011030411,),
 (2011030413,),
 (2011030415,),
 (2010030311,),
 (2010030313,),
 (2010030315,),
 (2010030317,)]

# Exploring possiblity of using json data from nhl api
https://steviehoward.com/NHL-API-docs/?python#getgame
- getting game strength would be possible with this library: https://pythonhosted.org/nhlscrapi/nhlscrapi.games.html#module-nhlscrapi.games.cumstats

In [245]:
example_responses = {
  "copyright": "string",
  "gamePk": 2017020851,
  "link": "/api/v1/game/2017020851/feed/live",
  "metaData": {
    "wait": 10,
    "timeStamp": "20180211_054345"
  },
  "gameData": {
    "game": {
      "pk": 2017020851,
      "season": "20172018",
      "type": "R"
    },
    "datetime": {
      "dateTime": "2018-02-11T03:00:00Z",
      "endDateTime": "2018-02-11T05:30:42Z"
    },
    "status": {
      "abstractGameState": "Final",
      "codedGameState": "7",
      "detailedState": "Final",
      "statusCode": "7",
      "startTimeTBD": True
    },
    "teams": {
      "away": {
        "id": 28,
        "name": "San Jose Sharks",
        "link": "/api/v1/teams/28",
        "venue": {
          "name": "SAP Center at San Jose",
          "link": "/api/v1/venues/null",
          "city": "San Jose",
          "timeZone": {
            "id": "America/Los_Angeles",
            "offset": -8,
            "tz": "PST"
          }
        },
        "abbreviation": "SJS",
        "triCode": "SJS",
        "teamName": "Sharks",
        "locationName": "San Jose",
        "firstYearOfPlay": 1990,
        "division": {
          "id": 15,
          "name": "Pacific",
          "link": "/api/v1/divisions/15"
        },
        "conference": {
          "id": 5,
          "name": "Western",
          "link": "/api/v1/conferences/5"
        },
        "franchise": {
          "id": 29,
          "name": "Sharks",
          "link": "/api/v1/franchises/29"
        },
        "roster": {
          "roster": [
            {
              "person": {
                "id": 8466138,
                "fullName": "Joe Thornton",
                "link": "/api/v1/people/8466138"
              },
              "jerseyNumber": 19,
              "position": {
                "code": "C",
                "name": "Center",
                "type": "Forward",
                "abbreviation": "C"
              }
            }
          ]
        },
        "nextGameSchedule": {
          "totalItems": 1,
          "totalEvents": 0,
          "totalGames": 1,
          "totalMatches": 0,
          "dates": [
            {
              "date": "2018-02-10",
              "totalItems": 1,
              "totalEvents": 0,
              "totalGames": 1,
              "totalMatches": 0,
              "games": [
                {
                  "gamePk": 2017020851,
                  "link": "/api/v1/game/2017020851/feed/live",
                  "gameType": "R",
                  "season": "20172018",
                  "gameDate": "2018-02-11T03:00:00Z",
                  "status": {
                    "abstractGameState": "Live",
                    "codedGameState": "3",
                    "detailedState": "In Progress",
                    "statusCode": "2",
                    "startTimeTBD": True
                  },
                  "teams": {
                    "away": {},
                    "home": {}
                  },
                  "venue": {
                    "name": "SAP Center at San Jose",
                    "link": "/api/v1/venues/null"
                  },
                  "content": {
                    "link": "/api/v1/game/2017020851/content"
                  }
                }
              ],
              "events": [
                {}
              ],
              "matches": [
                {}
              ]
            }
          ]
        },
        "shortName": "San Jose",
        "officialSiteUrl": "http://www.sjsharks.com",
        "franchiseId": 29,
        "active": True
      },
      "home": {
        "id": 28,
        "name": "San Jose Sharks",
        "link": "/api/v1/teams/28",
        "venue": {
          "name": "SAP Center at San Jose",
          "link": "/api/v1/venues/null",
          "city": "San Jose",
          "timeZone": {
            "id": "America/Los_Angeles",
            "offset": -8,
            "tz": "PST"
          }
        },
        "abbreviation": "SJS",
        "triCode": "SJS",
        "teamName": "Sharks",
        "locationName": "San Jose",
        "firstYearOfPlay": 1990,
        "division": {
          "id": 15,
          "name": "Pacific",
          "link": "/api/v1/divisions/15"
        },
        "conference": {
          "id": 5,
          "name": "Western",
          "link": "/api/v1/conferences/5"
        },
        "franchise": {
          "id": 29,
          "name": "Sharks",
          "link": "/api/v1/franchises/29"
        },
        "roster": {
          "roster": [
            {
              "person": {
                "id": 8466138,
                "fullName": "Joe Thornton",
                "link": "/api/v1/people/8466138"
              },
              "jerseyNumber": 19,
              "position": {
                "code": "C",
                "name": "Center",
                "type": "Forward",
                "abbreviation": "C"
              }
            }
          ]
        },
        "nextGameSchedule": {
          "totalItems": 1,
          "totalEvents": 0,
          "totalGames": 1,
          "totalMatches": 0,
          "dates": [
            {
              "date": "2018-02-10",
              "totalItems": 1,
              "totalEvents": 0,
              "totalGames": 1,
              "totalMatches": 0,
              "games": [
                {
                  "gamePk": 2017020851,
                  "link": "/api/v1/game/2017020851/feed/live",
                  "gameType": "R",
                  "season": "20172018",
                  "gameDate": "2018-02-11T03:00:00Z",
                  "status": {
                    "abstractGameState": "Live",
                    "codedGameState": "3",
                    "detailedState": "In Progress",
                    "statusCode": "2",
                    "startTimeTBD": True
                  },
                  "teams": {
                    "away": {},
                    "home": {}
                  },
                  "venue": {
                    "name": "SAP Center at San Jose",
                    "link": "/api/v1/venues/null"
                  },
                  "content": {
                    "link": "/api/v1/game/2017020851/content"
                  }
                }
              ],
              "events": [
                {}
              ],
              "matches": [
                {}
              ]
            }
          ]
        },
        "shortName": "San Jose",
        "officialSiteUrl": "http://www.sjsharks.com",
        "franchiseId": 29,
        "active": True
      }
    },
    "players": {
      "id": 8466138,
      "fullName": "Joe Thornton",
      "link": "/api/v1/people/8466138",
      "firstName": "Joe",
      "lastName": "Thornton",
      "primaryNumber": "19",
      "birthDate": "1979-07-02",
      "currentAge": 38,
      "birthCity": "London",
      "birthStateProvince": "ON",
      "birthCountry": "CAN",
      "nationality": "CAN",
      "height": "6' 4\"",
      "weight": 220,
      "active": True,
      "alternateCaptain": True,
      "captain": True,
      "rookie": True,
      "shootsCatches": "L",
      "rosterStatus": "I",
      "currentTeam": {
        "id": 28,
        "name": "San Jose Sharks",
        "link": "/api/v1/teams/28"
      },
      "primaryPosition": {
        "code": "C",
        "name": "Center",
        "type": "Forward",
        "abbreviation": "C"
      }
    },
    "venue": {
      "name": "SAP Center at San Jose",
      "link": "/api/v1/venues/null"
    }
  },
  "liveData": {
    "plays": {
      "allPlays": [
        {
          "players": [
            {
              "player": {
                "id": 8476881,
                "fullName": "Tomas Hertl",
                "link": "/api/v1/people/8476881"
              },
              "playerType": "Winner"
            }
          ],
          "result": {
            "event": "Game End",
            "eventCode": "SJS505",
            "eventTypeId": "GAME_END",
            "description": "Game End"
          },
          "about": {
            "eventIdx": 315,
            "eventId": 505,
            "period": 3,
            "periodType": "REGULAR",
            "ordinalNum": "3rd",
            "periodTime": "20:00",
            "periodTimeRemaining": "00:00",
            "dateTime": "2018-02-11T05:30:42Z",
            "goals": {
              "away": 4,
              "home": 6
            }
          },
          "coordinates": {
            "x": 0,
            "y": 0
          },
          "team": {
            "id": 28,
            "name": "San Jose Sharks",
            "link": "/api/v1/teams/28",
            "triCode": "SJS"
          }
        }
      ],
      "scoringPlays": [
        0
      ],
      "penaltyPlays": [
        0
      ],
      "playsByPeriod": [
        {
          "startIndex": 0,
          "plays": [
            0
          ],
          "endIndex": 114
        }
      ],
      "currentPlay": {
        "players": [
          {
            "player": {
              "id": 8476881,
              "fullName": "Tomas Hertl",
              "link": "/api/v1/people/8476881"
            },
            "playerType": "Winner"
          }
        ],
        "result": {
          "event": "Game End",
          "eventCode": "SJS505",
          "eventTypeId": "GAME_END",
          "description": "Game End"
        },
        "about": {
          "eventIdx": 315,
          "eventId": 505,
          "period": 3,
          "periodType": "REGULAR",
          "ordinalNum": "3rd",
          "periodTime": "20:00",
          "periodTimeRemaining": "00:00",
          "dateTime": "2018-02-11T05:30:42Z",
          "goals": {
            "away": 4,
            "home": 6
          }
        },
        "coordinates": {
          "x": 0,
          "y": 0
        },
        "team": {
          "id": 28,
          "name": "San Jose Sharks",
          "link": "/api/v1/teams/28",
          "triCode": "SJS"
        }
      }
    },
    "linescore": {
      "currentPeriod": 3,
      "currentPeriodOrdinal": "3rd",
      "currentPeriodTimeRemaining": "Final",
      "periods": [
        {
          "periodType": "REGULAR",
          "startTime": "2018-02-11T03:09:50Z",
          "endTime": "2018-02-11T03:44:47Z",
          "num": 1,
          "ordinalNum": "1st",
          "home": {
            "goals": 2,
            "shotsOnGoal": 14,
            "rinkSide": "left"
          },
          "away": {
            "goals": 0,
            "shotsOnGoal": 9,
            "rinkSide": "right"
          }
        }
      ],
      "shootoutInfo": {
        "away": {
          "scores": 0,
          "attempts": 0
        },
        "home": {
          "scores": 0,
          "attempts": 0
        }
      },
      "teams": {
        "home": {
          "team": {
            "id": 28,
            "name": "San Jose Sharks",
            "link": "/api/v1/teams/28",
            "abbreviation": "SJS",
            "triCode": "SJS"
          },
          "goals": 6,
          "shotsOnGoal": 30,
          "goaliePulled": True,
          "numSkaters": 5,
          "powerPlay": True
        },
        "away": {
          "team": {
            "id": 28,
            "name": "San Jose Sharks",
            "link": "/api/v1/teams/28",
            "abbreviation": "SJS",
            "triCode": "SJS"
          },
          "goals": 6,
          "shotsOnGoal": 30,
          "goaliePulled": True,
          "numSkaters": 5,
          "powerPlay": True
        }
      },
      "powerPlayStrength": "Even",
      "hasShootout": True,
      "intermissionInfo": {
        "intermissionTimeRemaining": 0,
        "intermissionTimeElapsed": 0,
        "inIntermission": True
      },
      "powerPlayInfo": {
        "situationTimeRemaining": 0,
        "situationTimeElapsed": 72,
        "inSituation": True
      }
    },
    "boxscore": {
      "teams": {
        "away": {
          "team": {
            "id": 28,
            "name": "San Jose Sharks",
            "link": "/api/v1/teams/28",
            "abbreviation": "SJS",
            "triCode": "SJS"
          },
          "teamStats": {
            "teamSkaterStats": {
              "goals": 6,
              "pim": 6,
              "shots": 30,
              "powerPlayPercentage": "0.0",
              "powerPlayGoals": 0,
              "powerPlayOpportunities": 1,
              "faceOffWinPercentage": "59.3",
              "blocked": 21,
              "takeaways": 9,
              "giveaways": 6,
              "hits": 15
            }
          },
          "players": {
            "person": {
              "id": 8471709,
              "fullName": "Marc-Edouard Vlasic",
              "link": "/api/v1/people/8471709",
              "shootsCatches": "L",
              "rosterStatus": "Y"
            },
            "jerseyNumber": "44",
            "position": {
              "code": "D",
              "name": "Defenseman",
              "type": "Defenseman",
              "abbreviation": "D"
            },
            "stats": {
              "skaterStats": {
                "timeOnIce": "23:04",
                "assists": 0,
                "goals": 0,
                "shots": 2,
                "hits": 0,
                "powerPlayGoals": 0,
                "powerPlayAssists": 0,
                "penaltyMinutes": 0,
                "faceOffWins": 0,
                "faceoffTaken": 0,
                "takeaways": 0,
                "giveaways": 1,
                "shortHandedGoals": 0,
                "shortHandedAssists": 0,
                "blocked": 0,
                "plusMinus": 1,
                "evenTimeOnIce": "18:12",
                "powerPlayTimeOnIce": "1:07",
                "shortHandedTimeOnIce": "3:45"
              }
            }
          },
          "goalies": [
            0
          ],
          "skaters": [
            0
          ],
          "onIce": [
            0
          ],
          "onIcePlus": [
            {
              "playerId": 8477180,
              "shiftDuration": 458,
              "stamina": 33
            }
          ],
          "scratches": [
            0
          ],
          "penaltyBox": [
            0
          ],
          "coaches": [
            {
              "person": {
                "fullName": "Peter DeBoer",
                "link": "/api/v1/people/null"
              },
              "position": {
                "code": "HC",
                "name": "Head Coach",
                "type": "Head Coach",
                "abbreviation": "Head Coach"
              }
            }
          ]
        },
        "home": {
          "team": {
            "id": 28,
            "name": "San Jose Sharks",
            "link": "/api/v1/teams/28",
            "abbreviation": "SJS",
            "triCode": "SJS"
          },
          "teamStats": {
            "teamSkaterStats": {
              "goals": 6,
              "pim": 6,
              "shots": 30,
              "powerPlayPercentage": "0.0",
              "powerPlayGoals": 0,
              "powerPlayOpportunities": 1,
              "faceOffWinPercentage": "59.3",
              "blocked": 21,
              "takeaways": 9,
              "giveaways": 6,
              "hits": 15
            }
          },
          "players": {
            "person": {
              "id": 8471709,
              "fullName": "Marc-Edouard Vlasic",
              "link": "/api/v1/people/8471709",
              "shootsCatches": "L",
              "rosterStatus": "Y"
            },
            "jerseyNumber": "44",
            "position": {
              "code": "D",
              "name": "Defenseman",
              "type": "Defenseman",
              "abbreviation": "D"
            },
            "stats": {
              "skaterStats": {
                "timeOnIce": "23:04",
                "assists": 0,
                "goals": 0,
                "shots": 2,
                "hits": 0,
                "powerPlayGoals": 0,
                "powerPlayAssists": 0,
                "penaltyMinutes": 0,
                "faceOffWins": 0,
                "faceoffTaken": 0,
                "takeaways": 0,
                "giveaways": 1,
                "shortHandedGoals": 0,
                "shortHandedAssists": 0,
                "blocked": 0,
                "plusMinus": 1,
                "evenTimeOnIce": "18:12",
                "powerPlayTimeOnIce": "1:07",
                "shortHandedTimeOnIce": "3:45"
              }
            }
          },
          "goalies": [
            0
          ],
          "skaters": [
            0
          ],
          "onIce": [
            0
          ],
          "onIcePlus": [
            {
              "playerId": 8477180,
              "shiftDuration": 458,
              "stamina": 33
            }
          ],
          "scratches": [
            0
          ],
          "penaltyBox": [
            0
          ],
          "coaches": [
            {
              "person": {
                "fullName": "Peter DeBoer",
                "link": "/api/v1/people/null"
              },
              "position": {
                "code": "HC",
                "name": "Head Coach",
                "type": "Head Coach",
                "abbreviation": "Head Coach"
              }
            }
          ]
        }
      },
      "officials": [
        {
          "official": {
            "id": 2071,
            "fullName": "Tim Peel",
            "link": "/api/v1/people/2071"
          },
          "officialType": "Linesman"
        }
      ]
    },
    "decisions": {
      "winner": {
        "id": 8477180,
        "fullName": "Aaron Dell",
        "link": "/api/v1/people/8477180"
      },
      "loser": {
        "id": 8477180,
        "fullName": "Aaron Dell",
        "link": "/api/v1/people/8477180"
      },
      "firstStar": {
        "id": 8477180,
        "fullName": "Aaron Dell",
        "link": "/api/v1/people/8477180"
      },
      "secondStar": {
        "id": 8477180,
        "fullName": "Aaron Dell",
        "link": "/api/v1/people/8477180"
      },
      "thirdStar": {
        "id": 8477180,
        "fullName": "Aaron Dell",
        "link": "/api/v1/people/8477180"
      }
    }
  }
}

In [246]:
example_responses.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [247]:
example_responses['gameData'].keys()

dict_keys(['game', 'datetime', 'status', 'teams', 'players', 'venue'])

In [248]:
example_responses['gameData']['status']

{'abstractGameState': 'Final',
 'codedGameState': '7',
 'detailedState': 'Final',
 'statusCode': '7',
 'startTimeTBD': True}

In [250]:
example_responses['liveData'].keys()

dict_keys(['plays', 'linescore', 'boxscore', 'decisions'])

In [252]:
example_responses['liveData']['linescore']

{'currentPeriod': 3,
 'currentPeriodOrdinal': '3rd',
 'currentPeriodTimeRemaining': 'Final',
 'periods': [{'periodType': 'REGULAR',
   'startTime': '2018-02-11T03:09:50Z',
   'endTime': '2018-02-11T03:44:47Z',
   'num': 1,
   'ordinalNum': '1st',
   'home': {'goals': 2, 'shotsOnGoal': 14, 'rinkSide': 'left'},
   'away': {'goals': 0, 'shotsOnGoal': 9, 'rinkSide': 'right'}}],
 'shootoutInfo': {'away': {'scores': 0, 'attempts': 0},
  'home': {'scores': 0, 'attempts': 0}},
 'teams': {'home': {'team': {'id': 28,
    'name': 'San Jose Sharks',
    'link': '/api/v1/teams/28',
    'abbreviation': 'SJS',
    'triCode': 'SJS'},
   'goals': 6,
   'shotsOnGoal': 30,
   'goaliePulled': True,
   'numSkaters': 5,
   'powerPlay': True},
  'away': {'team': {'id': 28,
    'name': 'San Jose Sharks',
    'link': '/api/v1/teams/28',
    'abbreviation': 'SJS',
    'triCode': 'SJS'},
   'goals': 6,
   'shotsOnGoal': 30,
   'goaliePulled': True,
   'numSkaters': 5,
   'powerPlay': True}},
 'powerPlayStrength'

## Trying to make call myself

In [274]:
import requests
headers = {
  'Accept': 'application/json'
}
id1 = '2017020851'
r = requests.get(f'https://statsapi.web.nhl.com/api/v1/game/{id1}/feed/live', params={
    }, headers = headers)
game1 = r.json()
assert game1

In [276]:
game1.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [308]:
game1['liveData']['plays']['allPlays'][0].keys()

dict_keys(['result', 'about', 'coordinates'])

In [310]:
game1['liveData']['plays']['allPlays'][0]['about']

{'eventIdx': 0,
 'eventId': 1,
 'period': 1,
 'periodType': 'REGULAR',
 'ordinalNum': '1st',
 'periodTime': '00:00',
 'periodTimeRemaining': '20:00',
 'dateTime': '2018-02-11T00:43:32Z',
 'goals': {'away': 0, 'home': 0}}

In [307]:
# len(game1['liveData']['plays'])
# len(game1['liveData']['plays']['allPlays'])
for x in game1['liveData']['plays']['allPlays']:
    if x['result']['event'] == 'Penalty':
        print(x)

{'players': [{'player': {'id': 8478414, 'fullName': 'Timo Meier', 'link': '/api/v1/people/8478414'}, 'playerType': 'PenaltyOn'}, {'player': {'id': 8477934, 'fullName': 'Leon Draisaitl', 'link': '/api/v1/people/8477934'}, 'playerType': 'DrewBy'}], 'result': {'event': 'Penalty', 'eventCode': 'SJS111', 'eventTypeId': 'PENALTY', 'description': 'Timo Meier Hooking against Leon Draisaitl', 'secondaryType': 'Hooking', 'penaltySeverity': 'Minor', 'penaltyMinutes': 2}, 'about': {'eventIdx': 25, 'eventId': 111, 'period': 1, 'periodType': 'REGULAR', 'ordinalNum': '1st', 'periodTime': '03:19', 'periodTimeRemaining': '16:41', 'dateTime': '2018-02-11T03:15:34Z', 'goals': {'away': 0, 'home': 1}}, 'coordinates': {'x': -53.0, 'y': 16.0}, 'team': {'id': 28, 'name': 'San Jose Sharks', 'link': '/api/v1/teams/28', 'triCode': 'SJS'}}
{'players': [{'player': {'id': 8476915, 'fullName': 'Jujhar Khaira', 'link': '/api/v1/people/8476915'}, 'playerType': 'PenaltyOn'}, {'player': {'id': 8476331, 'fullName': 'Dyla

In [315]:
for x in game1['liveData']['plays']['allPlays']:
#     if x['result']['event'] == 'Penalty':
        if x['about']['period'] == 1:
            if x['about']['periodTime'][0:2] == '03' or x['about']['periodTime'][0:2] == '04':
                pprint.pprint(x)

{'about': {'dateTime': '2018-02-11T03:14:38Z',
           'eventId': 17,
           'eventIdx': 21,
           'goals': {'away': 0, 'home': 1},
           'ordinalNum': '1st',
           'period': 1,
           'periodTime': '03:03',
           'periodTimeRemaining': '16:57',
           'periodType': 'REGULAR'},
 'coordinates': {'x': 39.0, 'y': -23.0},
 'players': [{'player': {'fullName': 'Justin Braun',
                         'id': 8474027,
                         'link': '/api/v1/people/8474027'},
              'playerType': 'Shooter'},
             {'player': {'fullName': 'Al Montoya',
                         'id': 8471219,
                         'link': '/api/v1/people/8471219'},
              'playerType': 'Goalie'}],
 'result': {'description': 'Justin Braun Slap Shot saved by Al Montoya',
            'event': 'Shot',
            'eventCode': 'SJS17',
            'eventTypeId': 'SHOT',
            'secondaryType': 'Slap Shot'},
 'team': {'id': 28,
          'link': '/api/v1/